In [ ]:
import time
import subprocess
import vendor.lightning_pb2 as ln
import vendor.lightning_pb2_grpc as lnrpc
import vendor.peers_pb2_grpc as peers
import vendor.router_pb2_grpc as router
import grpc
import codecs
import hashlib


AGENT_ID = 20235

class LndClient:
    def __init__(self, host, port, macaroon_path, tls_cert_path):
        self.host = host
        self.port = port
        self.macaroon_path = macaroon_path
        self.tls_cert_path = tls_cert_path

        self.channel = self.build()
        self.lightning_stub = lnrpc.LightningStub(self.channel)
        self.peers_stub = peers.PeersStub(self.channel)
        self.router_stub = router.RouterStub(self.channel)

    def build(self):
        tls_cert = open(self.tls_cert_path, "rb").read()
        macaroon = codecs.encode(open(self.macaroon_path, "rb").read(), "hex")
        cert_creds = grpc.ssl_channel_credentials(tls_cert)
        auth_creds = grpc.metadata_call_credentials(self._macaroon_call(macaroon))
        creds = grpc.composite_channel_credentials(cert_creds, auth_creds)
        channel = grpc.secure_channel(f"{self.host}:{self.port}", creds)
        return channel


    def _macaroon_call(self, macaroon):
        def metadata_callback(context, callback):
            callback([('macaroon', macaroon)], None)
        return metadata_callback

def build_lnd1():
    host = "127.0.0.1"
    port = 30009
    
    macaroon_path='../volumes/lnd/data/chain/bitcoin/regtest/admin.macaroon'
    tls_cert_path='../volumes/lnd/tls.cert'
    
    return LndClient(host, port, macaroon_path, tls_cert_path)

def build_lnd2():
    host = "127.0.0.1"
    port = 30010
    
    macaroon_path='../volumes/lnd2/data/chain/bitcoin/regtest/admin.macaroon' 
    tls_cert_path='../volumes/lnd2/tls.cert'

    return LndClient(host, port, macaroon_path, tls_cert_path)

